In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import pandas as pd
import warnings
import math
import random
warnings.filterwarnings('ignore')
import timeit
data = pd.read_csv("C:/Users/Mehmet/Desktop/spambase.data",header = None,sep=',')


In [2]:
data.head(6)

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
5,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.00,0.223,0.0,0.000,0.000,0.000,3.000,15,54,1


In [3]:
X = data.iloc[:, 0:56].values
y = data.iloc[:, 57].values

In [4]:
X[0]


array([ 0.   ,  0.64 ,  0.64 ,  0.   ,  0.32 ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.64 ,  0.   ,  0.   ,  0.   ,  0.32 ,
        0.   ,  1.29 ,  1.93 ,  0.   ,  0.96 ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.778,  0.   ,  0.   ,  3.756, 61.   ])

In [5]:
y[:6]

array([1, 1, 1, 1, 1, 1], dtype=int64)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
print(y.shape)
print(y_train.shape)

(4601,)
(3450,)


In [7]:

class agent():
    def __init__(self,ID, katmanlar=[]):
        self.katmanlar = katmanlar
        self.ID=ID
        self.H = [] 
        self.W=[]
        self.onlyOnce = True
    
    def feed(self,x=X_train,y=y_train):
        self.Y=y
        self.X=x
        self.b = [np.random.randn(k, 1) for k in self.katmanlar[1:]] 
        self.W = [5*np.random.randn(k2, k1) for k1, k2 in zip(self.katmanlar[:-1],self.katmanlar[1:])]
        self.score = self.performance()
   
    
    def predict(self, X_test):
        if self.katmanlar[-1] == 1 : 
            tahmin = self.ileribesleme(X_test.T) >= 0.5  
            t = tahmin.astype('int')
            return t[0]
        return np.argmax(self.ileribesleme(X_test.T), axis= 0)
    
    
    
    def immitate(self,other):
        
            l =  random.randint(0,(self.katmanlar[-2]-1))#11
            p=0;
            for i in self.katmanlar[1:-1]:
                k =  random.randint(0,(i-1))
                self.b[p][k]=other.b[p][k]
                self.W[p][k]=other.W[p][k]
                p=p+1
            self.W[-1][0][l]=other.W[-1][0][l]
            self.score = self.performance()
            
            
    def innovate(self, pr = 0.1):
        if np.random.rand() < pr: 

            
            l =  random.randint(0,(self.katmanlar[-2]-1))#11
            p=0;
            for i in self.katmanlar[1:-1]:
                k =  random.randint(0,(i-1))
                self.b[p][k]=np.random.randn()
                self.W[p][k]=5*np.random.randn()
                p=p+1
            self.W[-1][0][l]=5*np.random.randn()
            
            self.score = self.performance()
    
    
    
    def performance(self,threshold=0.1):
        
        abm_pred=self.ileribesleme() 
        p = abm_pred >= threshold
        scoree=1/(1+np.sum(np.power(self.Y - p,2)))
        
        return scoree
        
    
    def ileribesleme(self, a=X_train.T):

        a = self.checkDimension(a)
        
        for w, b in zip(self.W, self.b):
            z = np.dot(w, a)+b
            a = self.sigmoid(z)
        
        return a
    
    
    def sigmoid(self,z):
        import math

        return 1.0/(1.0+np.exp(-z))
    
    
    def relu(self,x):
        return (max(0, np.any(x)))
    
    def checkDimension(self,x):
        if x.ndim == 1: return x.reshape(x.shape[0], 1)
        return x
              
class abmNeural():
    def __init__(self, X=X_train, y=y_train, N = 100, time = 1000,katmanlar=[]):
        self.X, self.y, self.N, self.time,self.katmanlar = X, y, N, time,katmanlar
        kts=katmanlar
        self.population = [agent(i,kts) for i in range(self.N)]
        
    def feed(self):
        for A in self.population:
            A.feed(self.X, self.y) 
    
    def social_optimisation(self):
        #self.feed()
        for i in range(self.time):
            iA, iB = np.random.choice(range(self.N), 2, replace=False)
            A, B = self.population[iA], self.population[iB]
            
            if A.score > B.score: 
                B.immitate(A)
                A.innovate();B.innovate(pr=0.3)
                
            else: 
                A.immitate(B)
                A.innovate(pr=0.3);B.innovate()
                           

    
    def social_optimisation2(self):#choose based probability
        #self.feed()
        scores = [A.score for A in self.population]
        scores=np.asarray(scores)
        s_sum=scores.sum()
        
        for i in range(self.time):
            iA, iB = np.random.choice(range(self.N), 2, replace=False,p=scores/s_sum)
            A, B = self.population[iA], self.population[iB]
            
            if A.score > B.score: 
                B.immitate(A)
                A.innovate()
                B.innovate(pr=0.3)
                
            else: 
                A.immitate(B)
                B.innovate()
                A.innovate(pr=0.3)
                
                
    def social_optimisationDevam(self):#Kills half of the worst population and replaces them with new generation
        #self.feed()
        
        pop=[A.score for A in self.population]
        newpop=[]   
        k=int(self.N/2)
        for i in range(int(k)):
            m=int(pop.index(max(pop)))
            newpop.append(self.population[m])
            pop[m]=0
        
        
        newpop[k:]=[agent(i,self.katmanlar) for i in range(k)]
        
        for A in newpop[k:]:
            A.feed()
        
        self.population=newpop
        
        
        self.social_optimisation2()
                
    def social_optimisation3(self,N=10):
#runs socialoptimisationDevam N times, kills half of the population and runs socialoptimisation N/2 times so imitation occurs 
# in best N/2 agents over generations
        for i in range(N):
            self.social_optimisationDevam()
        
        pop=[A.score for A in self.population]
        newpop=[]   
        k=int(self.N/2)
        for i in range(int(k)):
            m=int(pop.index(max(pop)))
            newpop.append(self.population[m])
            pop[m]=0
        
        self.population=newpop
        
        self.N=int(self.N/2)
        
        for i in range(int(N/2)):
            self.social_optimisation2()
            
    def best_agent(self):   
        scores = [A.score for A in self.population]
        ibest = scores.index(max(scores))
        return self.population[ibest]
        
    
    def scores(self):   
        scores = [A.score for A in self.population]
        return scores

    
    def predict(self, X_test=X_test.T, threshold = 0.5):
        
        res=self.best_agent().predict(X_test=X_test)
        m,n=X_test.shape
        
        return res
    


In [8]:
abm= abmNeural(X=X_train, y=y_train, N = 1000, time = 10000,katmanlar=[56,1])
abm.feed()


#-------------------------------------------------------------------------------------------------------------------------------




In [9]:
abm_predWithoutImitation=abm.predict(X_test)
abm.social_optimisation()
abm_pred=abm.predict(X_test)
print(abm_pred.shape)

(1151,)


In [10]:
abm2= abmNeural(X=X_train, y=y_train, N = 1000, time = 10000,katmanlar=[56,1])
abm2.population=abm.population

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, abm_pred)
print("\t\t\t\t---Our Own Neural Network with ABM---")
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(y_test, abm_pred))


				---Our Own Neural Network with ABM---
confusion_matrix:
 [[456 235]
 [ 92 368]]
accuracy_score:  0.7158992180712423


In [12]:
abm.social_optimisation3(N=10)
abm_pred=abm.predict(X_test)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, abm_pred)
print("\t\t\t\t---Our Own Neural Network with ABMv3---")
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(y_test, abm_pred))


				---Our Own Neural Network with ABMv3---
confusion_matrix:
 [[611  80]
 [169 291]]
accuracy_score:  0.7836663770634231


In [13]:
abm2.social_optimisation2()
abm_predv3=abm2.predict(X_test)

In [14]:
print("\t\t\t\t---Our Own Neural Network without imitation---")
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(y_test, abm_predWithoutImitation))
print("\t\t\t\t---Our Own Neural Network with ABMv2---")
print("confusion_matrix:\n", cm)
print("accuracy_score: ", accuracy_score(y_test, abm_predv3))

				---Our Own Neural Network without imitation---
confusion_matrix:
 [[611  80]
 [169 291]]
accuracy_score:  0.7098175499565595
				---Our Own Neural Network with ABMv2---
confusion_matrix:
 [[611  80]
 [169 291]]
accuracy_score:  0.7445699391833188
